# Preparation

In [ ]:
import datetime
import pandas as pd
from tqdm.notebook import tqdm
import requests
import time

In [ ]:
# rank_df = pd.read_csv("../input/rps-1129-rank/rock-paper-scissors-publicleaderboard.csv")
# rank_df = rank_df.sort_values("Score", ascending=False)
# rank_df.head()

# list_url = 'https://www.kaggle.com/requests/EpisodeService/ListEpisodes'
# episode_list = list()

# for tid, tname in tqdm(rank_df[["TeamId", "TeamName"]].values):
    
#     t = requests.post(list_url, json={'TeamId': tid})
    
#     # Too Many Requests
#     # Inevitable
#     if t.status_code == 429:
#         print(tid)
#         break
    
#     t = t.json()
        
    
#     for ep in t["result"]["episodes"]:
#         try:
#             create_time_second = ep['createTime']["seconds"]
#             create_time_nano = ep['createTime']["nanos"]
#             end_time_second = ep['endTime']["seconds"]
#             end_time_nano = ep['endTime']["nanos"]

#             player_1_submission_id = ep['agents'][0]["submissionId"]
#             player_1_init_score = ep['agents'][0]["initialScore"]
#             player_1_update_score = ep["agents"][0]["updatedScore"]
#             player_2_submission_id = ep['agents'][1]["submissionId"]
#             player_2_init_score = ep['agents'][1]["initialScore"]
#             player_2_update_score = ep["agents"][1]["updatedScore"]


#             episode_list.append([
#                 create_time_second,
#                 create_time_nano,
#                 end_time_second,
#                 end_time_nano,
#                 player_1_submission_id,
#                 player_1_init_score,
#                 player_1_update_score,
#                 player_2_submission_id,
#                 player_2_init_score,
#                 player_2_update_score,
#             ])

#         except:
#             print(tid)
#             break
        
# episode_df = pd.DataFrame(episode_list, columns=["createTimeSecond", "createTimeNano", "endTimeSecond", "endTimeNano", "Player1SID", "Player1InitScore", "Player1UpdateScore", "Player2SID", "Player2InitScore", "Player2UpdateScore"])

In [ ]:
# ALL GAMES BEFORE 29 Nov 2020
# Some may be missing.
episode = pd.concat([
    pd.read_csv("../input/episode1129/episode_1.csv"),
    pd.read_csv("../input/episode1129/episode_2.csv"),
    pd.read_csv("../input/episode1129/episode_3.csv"),
    pd.read_csv("../input/episode1129/episode_4.csv"),
    pd.read_csv("../input/episode1129/episode_5.csv"),
    pd.read_csv("../input/episode1129/episode_6.csv"),
], axis=0)

In [ ]:
# print(len(episode))
episode = episode[~episode.duplicated()].reset_index(drop=True)
# print(len(episode))
episode_change_12 = episode.copy()
episode_change_12.columns = [
    'createTimeSecond', 
    'createTimeNano', 
    'endTimeSecond', 
    'endTimeNano',
    'Player2SID', 
    'Player2InitScore', 
    'Player2UpdateScore', 
    'Player1SID',
    'Player1InitScore', 
    'Player1UpdateScore'
]
episode = pd.concat([episode, episode_change_12], axis=0)
# print(len(episode))
episode = episode[~episode.duplicated()].reset_index(drop=True)
# print(len(episode))

In [ ]:
episode["createTimeSecond"] = episode["createTimeSecond"].apply(datetime.datetime.fromtimestamp)
episode["endTimeSecond"] = episode["endTimeSecond"].apply(datetime.datetime.fromtimestamp)
episode = episode.sort_values(["Player1SID", "endTimeSecond"])
episode = episode[episode["createTimeSecond"] < "2020-11-29"].reset_index(drop=True)

# Summarize Before and After Update Top 100

Compare Top 100 agent before and before updating rating system.

Rating system is changed in 16~17 Nov.

In [ ]:
# 16 Nov 2020
bfr_update_top100_submit = episode[episode["endTimeSecond"] < "2020-11-17"].groupby("Player1SID")["Player1UpdateScore"].apply(
    lambda x: x.tail(1).values[0]
).to_frame().sort_values("Player1UpdateScore", ascending=False).head(100)
bfr_update_top100_submit

In [ ]:
bfr_top100_submit_id = bfr_update_top100_submit.index.tolist()
bfr_update_top100_history = episode[episode["Player1SID"].isin(bfr_top100_submit_id)].pivot_table(index="endTimeSecond", columns="Player1SID", values="Player1UpdateScore")

In [ ]:
# 29 Nov 2020
aft_update_top100_submit = episode.groupby("Player1SID")["Player1UpdateScore"].apply(
    lambda x: x.tail(1).values[0]
).to_frame().sort_values("Player1UpdateScore", ascending=False).head(100)
aft_update_top100_submit

## Top 100 just before the update

In [ ]:
aft_top100_submit_id = aft_update_top100_submit.index.tolist()
aft_update_top100_history = episode[episode["Player1SID"].isin(aft_top100_submit_id)].pivot_table(index="endTimeSecond", columns="Player1SID", values="Player1UpdateScore")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(figsize=(20, 10))

for sub in bfr_update_top100_history.columns:
    if sub in aft_update_top100_history.columns:
        bfr_update_top100_history[bfr_update_top100_history[sub].notnull()][sub].plot(ax=ax, color="red")
    else:
        bfr_update_top100_history[bfr_update_top100_history[sub].notnull()][sub].plot(ax=ax, color="grey")
plt.vlines([datetime.datetime(2020, 11, 17, 0, 0, 0)], 500, 1300, "blue", linestyle="dashed")
plt.hlines([950.781712], datetime.datetime(2020, 11, 1, 0, 0, 0), datetime.datetime(2020, 11, 29, 0, 0, 0), "blue", linestyle="dashed")
plt.title("Top 100 Rate just before the update(Red: Now Top 100, Grey: Other)")

In [ ]:
print("Mean Rate of Top 100 Before Update", bfr_update_top100_submit["Player1UpdateScore"].mean())
print("Mean Now Rate of Top 100 Rate Before Update",episode[episode["Player1SID"].isin(bfr_top100_submit_id)].groupby("Player1SID")["Player1UpdateScore"].apply(lambda x: x.head(1).values[0]).mean())

The score of the agent who was in the top 100 just before the update has dropped by about 250 points since then.

## Now(29 Nov) Top 100

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(figsize=(20, 10))

# update date
plt.vlines([datetime.datetime(2020, 11, 17, 0, 0, 0)], 500, 1300,  "blue" , linestyle="dashed")

# before update date rank 100 score
plt.hlines([1049.045015], datetime.datetime(2020, 11, 1, 0, 0, 0), datetime.datetime(2020, 11, 29, 0, 0, 0), "blue", linestyle="dashed")
for sub in aft_update_top100_history.columns:
    if sub in bfr_update_top100_history.columns:
        aft_update_top100_history[aft_update_top100_history[sub].notnull()][sub].plot(ax=ax, color='r')
    else:
        aft_update_top100_history[aft_update_top100_history[sub].notnull()][sub].plot(ax=ax, color='grey')
plt.title("Now Top 100 Rate(Red: Top 100 just before the update, Grey: Other)")

In [ ]:
aft_update_top_100_create_date = episode[episode["Player1SID"].isin(aft_top100_submit_id)].groupby("Player1SID")["createTimeSecond"].min().reset_index()
aft_update_top_100_create_date["is_before_update_top_100"] = aft_update_top_100_create_date["Player1SID"].apply(lambda x: (x in bfr_top100_submit_id)*1)
aft_update_top_100_create_date["is_before_update_submit"] = aft_update_top_100_create_date["createTimeSecond"].apply(lambda x: x < datetime.datetime(2020, 11, 17))
pd.crosstab(aft_update_top_100_create_date["is_before_update_submit"], aft_update_top_100_create_date["is_before_update_top_100"])

Only 18% of the top agents before the update are still in the top 100

50% agents submitted after update

# N1 analyze

In [ ]:
def plot_related_with_battle_and_score(base):
    fig, ax1 = plt.subplots(figsize=(20, 10))
    ax2 = ax1.twinx()

    ax1.bar(base["index"], base["Player1UpScore"], label="Up(win) or Down(lose) Rate")
    ax2.scatter(base["index"], base["Player1InitScore"], c="r", label="My Rate (right label)")
    ax2.scatter(base["index"], base["Player2InitScore"], c="blue", label="Opponent Rate (right label)")
    handler1, label1 = ax1.get_legend_handles_labels()
    handler2, label2 = ax2.get_legend_handles_labels()
    ax1.legend(handler1 + handler2, label1 + label2, loc=1, borderaxespad=0.)

In [ ]:
# Top 1 before update rate logic
n1_17901899 = episode[episode["Player1SID"] == 17901899].reset_index(drop=True)
n1_17901899["Player1UpScore"] = n1_17901899["Player1UpdateScore"] - n1_17901899["Player1InitScore"]
n1_17901899 = n1_17901899.reset_index()

In [ ]:
plot_related_with_battle_and_score(n1_17901899)

In [ ]:
# Top 1 after update rate logic
n1_18317942 = episode[episode["Player1SID"] == 18317942].reset_index(drop=True)
n1_18317942["Player1UpScore"] = n1_18317942["Player1UpdateScore"] - n1_18317942["Player1InitScore"]
n1_18317942 = n1_18317942.reset_index()

In [ ]:
plot_related_with_battle_and_score(n1_18317942)

The rate of decrease when defeated by a low-rate agent is too small?

The key to the competition is how lucky you can win in the first few games

# Rating System

## This Competition Rating System

https://www.kaggle.com/c/rock-paper-scissors/overview/evaluation

## Other Rating System

I don't know the optimal rating system, but there is something like this

https://en.wikipedia.org/wiki/Elo_rating_system

https://en.wikipedia.org/wiki/Glicko_rating_system

And more...